In [228]:
import requests
import json
import sys
import os

import asyncio
import aiofiles
import aiohttp

In [247]:
args = sys.argv
testScriptName = args[0]

print("Running ", args[0])

username = args[1] if len(args) > 1 else ""
password = args[2] if len(args) > 2 else ""

# authenticate
signin(username, password)

Running  /Users/temogen/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py
{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "password": "/Users/temogen/Library/Jupyter/runtime/kernel-13c927e2-d47f-425c-af83-fadc0e55b45b.json", 
    "username": "-f"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Content-Length": "120", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.18.4"
  }, 
  "json": null, 
  "origin": "73.51.189.184", 
  "url": "https://httpbin.org/post"
}

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.18.4"
  }, 
  "origin": "73.51.189.184", 
  "url": "https://httpbin.org/get"
}



In [246]:
def signin(username, password):
    payload = {
        'username': username,
        'password': password
    }

    with requests.Session() as s:
        p = s.post('https://httpbin.org/post', data=payload)
        print(p.text)

        r = s.get('https://httpbin.org/get')
        print(r.text)

In [215]:
execute = 'execute'
given = 'given'
expected = 'expected'
operation = 'operation'

testDirectory = './tests/'
authDirectory = './'

authfile = 'signinauth.json'

operations = {'GET': (lambda endpoint, request: requests.get(url=endpoint, params=request)), 
              'POST':(lambda endpoint, request: requests.post(url=endpoint, data=request)), 
              'PUT': (lambda endpoint, request: requests.put(url=endpoint, data=request)), 
              'DELETE':(lambda endpoint, request: requests.delete(url=endpoint, data=request)),
              'OPTIONS':(lambda endpoint, request: requests.options(url=endpoint))}

In [256]:
def run(testFiles):
    for testfile in testFiles:
        print(newTestingFile + "Executing ", testfile + endcomment)
        with open(testDirectory + testfile) as rawTests:
            tests = json.load(rawTests)
            executeTests(tests)
            
def retrieveTestFiles():
    testFiles = [posJson for posJson in os.listdir(testDirectory) if posJson.endswith('.json')]
    
    return testFiles

def executeTests(tests):
    for endpoint in tests[execute]:
        for test in tests[execute][endpoint]:
            print(test, 'hitting endpoint: ', endpoint)
            request = tests[execute][endpoint][test][given]
            opt = tests[execute][endpoint][test][operation]
            response = operations[opt.upper()](endpoint, request)
            expectedCode = tests[execute][endpoint][test][expected]
            success = (successfulFormat + "PASSED") if response.status_code == expectedCode else (failureFormat + "FAILED")
            print('\t' + test, success + bformat.CLOSE, response, 'received, ', expectedCode, "expected ")
            print('\treason: ', response.reason)

In [257]:
testFiles = retrieveTestFiles()
run(testFiles)

Executing  deleteTesting.json
test1 hitting endpoint:  https://httpbin.org/delete
	test1 PASSED <Response [405]> received,  405 expected 
	 reason:  METHOD NOT ALLOWED
test2 hitting endpoint:  https://httpbin.org/delete
	test2 PASSED <Response [200]> received,  200 expected 
	 reason:  OK
test3 hitting endpoint:  https://httpbin.org/delete
	test3 PASSED <Response [200]> received,  200 expected 
	 reason:  OK
Executing  postTesting.json
test1 hitting endpoint:  https://httpbin.org/post
	test1 PASSED <Response [200]> received,  200 expected 
	 reason:  OK
test2 hitting endpoint:  https://httpbin.org/post
	test2 PASSED <Response [405]> received,  405 expected 
	 reason:  METHOD NOT ALLOWED
test3 hitting endpoint:  https://httpbin.org/post
	test3 PASSED <Response [200]> received,  200 expected 
	 reason:  OK
test4 hitting endpoint:  https://httpbin.org/post
	test4 PASSED <Response [200]> received,  200 expected 
	 reason:  OK
test1 hitting endpoint:  https://httpbin.org/post2
	test1 PASSED

In [166]:
''' Example Structure

{
    "execute": {
        "<url>:<port?>/<api directory>/<endpoint>": {
            "<testname>": {
                "operation": "GET",
                "given": { "item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }, 
            "<testname2>": {
                "operation": "POST",
                "given": {"item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }
        }, 
        "<url>:<port?>/<api directory>/<endpoint2>": {
             "<testname1>": {
                "operation": "PUT",
                "given": { "item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }, 
            "<testname2>": {
                "operation": "DELETE",
                "given": {"item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }
        }
    }, 
    "skip": {
        "<url>:<port?>/<api directory>/<endpoint3>": {
            "<testname>": {
                "operation": "GET",
                "given": { "item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }, 
            "<testname2>": {
                "operation": "POST",
                "given": {"item1": "thing", "item2": 1}, 
                "expected":{"item1": "returned"}
            }
        }
    }
}

'''

' Example Structure\n\n{\n    "execute": {\n        "<url>:<port?>/<api directory>/<endpoint>": {\n            "<testname>": {\n                "operation": "GET",\n                "given": { "item1": "thing", "item2": 1}, \n                "expected":{"item1": "returned"}\n            }, \n            "<testname2>": {\n                "operation": "POST",\n                "given": {"item1": "thing", "item2": 1}, \n                "expected":{"item1": "returned"}\n            }\n        }, \n        "<url>:<port?>/<api directory>/<endpoint2>": {\n             "<testname1>": {\n                "operation": "PUT",\n                "given": { "item1": "thing", "item2": 1}, \n                "expected":{"item1": "returned"}\n            }, \n            "<testname2>": {\n                "operation": "DELETE",\n                "given": {"item1": "thing", "item2": 1}, \n                "expected":{"item1": "returned"}\n            }\n        }\n    }, \n    "skip": {\n        "<url>:<port?>/

In [197]:
class bformat:
    HEADER = '\033[95m'
    NEUTRAL = '\033[94m'
    SUCCESS = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    CLOSE = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
successfulFormat = bformat.SUCCESS
failureFormat = bformat.FAIL
newTestingFile = bformat.HEADER + bformat.BOLD
endcomment = bformat.CLOSE

In [258]:
!python ./run.py username password

{}
Running  ./run.py
{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "password": "password", 
    "username": "username"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Content-Length": "35", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.18.4"
  }, 
  "json": null, 
  "origin": "73.51.189.184", 
  "url": "https://httpbin.org/post"
}

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.18.4"
  }, 
  "origin": "73.51.189.184", 
  "url": "https://httpbin.org/get"
}

Executing  deleteTesting.json
test1 hitting endpoint:  https://httpbin.org/delete
	test1 PASSED <Response [405]> received,  405 expected 
	 reason {'Connection': 'keep-alive', 'Server': 'meinheld/0.6.1', 'Date': 'Mon, 11 Dec 2017 03:

	test3 PASSED <Response [404]> received,  404 expected 
	 reason {'Connection': 'keep-alive', 'Server': 'meinheld/0.6.1', 'Date': 'Mon, 11 Dec 2017 03:27:48 GMT', 'Content-Type': 'text/html', 'Content-Length': '233', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, PATCH, OPTIONS', 'Access-Control-Max-Age': '3600', 'X-Powered-By': 'Flask', 'X-Processed-Time': '0.000890016555786', 'Via': '1.1 vegur'}
Executing  putTesting.json
test1 hitting endpoint:  https://httpbin.org/put
	test1 PASSED <Response [200]> received,  200 expected 
	 reason {'Connection': 'keep-alive', 'Server': 'meinheld/0.6.1', 'Date': 'Mon, 11 Dec 2017 03:27:50 GMT', 'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true', 'X-Powered-By': 'Flask', 'X-Processed-Time': '0.000705003738403', 'Content-Length': '358', 'Via': '1.1 vegur'}
test1.1 hitting endpoint:  https://httpbin.